<a href="https://colab.research.google.com/github/pl98754321/POS-tagging-LST20/blob/main/W2N2P1_LST20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# initial Function

In [ ]:
! pip install -qq datasets transformers[sentencepiece] simpletransformers
! pip install -qq simpletransformers
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import re
import string
import os
from tqdm import tqdm
import glob

#Model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import torch
import joblib
from ast import literal_eval

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

#image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

#torch
import torch as T
import torch.nn as N
import torch.optim as O

In [ ]:
!nvidia-smi

Sun Feb 20 11:53:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    42W / 250W |   1583MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# get Data

LST20

In [ ]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
! tar -xvzf /content/AIFORTHAI-LST20Corpus.tar.gz

# get Embedding layer From HuggingFace


In [ ]:
from transformers import AutoTokenizer,AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')
model_hug = AutoModelForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased',
                                                            revision='finetuned@lst20-pos')


In [ ]:
word_em_initial = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
np_emb_weight = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocal_all = list(dict_word2int.keys()) #ใช้สำหรับตรวจคำ
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

PreTrainedTokenizerFast(name_or_path='airesearch/wangchanberta-base-att-spm-uncased', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']})

# Func Batchsize

In [ ]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

# loop map ค่าใน dict
def endcode_word(word):
  if word==" ": #clear ช่องว่างออกก่อน
    return 10

  elif word in dict_word2int: #ถ้าเจอใน ditionnary
    return dict_word2int[word]

  # else:
  #   return 3

  else: # ถ้าไม่เจอใน dictionary ก็ค่อยตัดคำ
    for i in tokenizer(word)["input_ids"]:
      if i not in [5,6,10]:
        return i

In [ ]:
def encode_label(text) :
  payload = {'NN' : 0, 'VV' : 1, 'CC' :2, 'PU' : 3, 'PS' : 4, 'AX' : 5, 'AV' : 6, 'FX' : 7, 'NU' : 8, 'AJ' : 9, 'CL' : 10, 'PR' : 11, 'NG' : 12, 'PA' :13, 'XX' :14, 'IJ' :15}
  return payload[text]
def decode_label(text) :
  payload =   {0: "NN",1: "VV",3: "CC",2: "PU",4: "PS",5: "AX",6: "AV",7: "FX",8: "NU",9: "AJ",10: "CL",11: "PR",12: "NG",13: "PA",14: "XX",15: "IJ"}
  return payload[text]

In [ ]:
_NER_TAGS = ["O","B_BRN","B_DES","B_DTM","B_LOC","B_MEA","B_NUM","B_ORG","B_PER","B_TRM","B_TTL","I_BRN","I_DES","I_DTM","I_LOC","I_MEA","I_NUM","I_ORG","I_PER","I_TRM","I_TTL","E_BRN","E_DES","E_DTM","E_LOC","E_MEA","E_NUM","E_ORG","E_PER","E_TRM","E_TTL",]
list_index = range(len(_NER_TAGS))
encode_NER_dict = dict(zip(_NER_TAGS,list_index))
decode_NER_dict = dict(zip(list_index,_NER_TAGS))
def encode_NER(word):
  if word in _NER_TAGS:
    return encode_NER_dict[word]
  return len(_NER_TAGS)
def decode_NER(num):
  if len(_NER_TAGS) == num:
    return "UNK"
  return decode_NER_dict[num]

In [ ]:
a = np.array(["a","b","c"])
d = np.array([1,2,3])

for i,j in a,d:
  print(i)
  print(j)

ValueError: ignored

In [ ]:
class get_data():
  def preprocessing(self,x,encode_dict,word=False):
    words_payload_encoded = [encode_dict(r) for r in tqdm(x)]
    np_word_encoded = np.array(words_payload_encoded)

    batched_word = np_word_encoded[:-1*(np_word_encoded.shape[0]%self.BATCH_SIZE)].reshape(-1,self.BATCH_SIZE)

    batched_word = batched_word.astype('float32')

    return batched_word

  def ADD_NER(self):
    self.NER = self.preprocessing(self.df["NER"].to_list(),encode_NER)
    
  def __init__(self,df,TAG,BATCH_SIZE):
    self.df = df
    self.BATCH_SIZE = BATCH_SIZE
    self.x = self.preprocessing(df["words"].to_list(),endcode_word,word=True)
    self.y = self.preprocessing(df[TAG].to_list(),encode_label)
    self.train = [[word,NER] for word,NER in ]

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self):
    return self.x , self.y

  def 

In [ ]:
class set_model():
  def __init__(self,model,train_get_data,val_get_data):
    self.model = model
    self.train = train_get_data
    self.val = val_get_data

  def train(self):
    ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
    self.model.fit(self.train.x,self.train.y, 
                   epochs=100,
                   validation_data=(self.val.x,self.val.x),
                   callbacks = [ES],
                   batch_size = 64)
    
  

In [ ]:
def LST20_list(path = '/content/LST20_Corpus/test/*'):
  sentense_payload = []
  words_payload = []
  POS_payload = []
  NER_payload = []
  index = 0
  for i in glob.glob(path) :
    fp = open(i,"r")
    for line in fp.readlines() :
      if line == '\n' :
        continue
      else :
        data_ = line[:-2].split("\t")
        sentense_payload.append(index)
        words_payload.append(data_[0])
        POS_payload.append(data_[1])
        NER_payload.append(data_[2])
    index +=1
  return pd.DataFrame(
      {"sentence_id": sentense_payload, "words": words_payload, "POS": POS_payload,"NER":NER_payload}
  )

นี่คือการทดลองทำ Batch 128

In [ ]:
BATCH_SIZE = 128

# LSTM

## data loader

In [ ]:
train_df = LST20_list(path = '/content/LST20_Corpus/train/*')
train_data = get_data(train_df,
                      "POS",
                      BATCH_SIZE)
train_data.ADD_NER()

In [ ]:
val_df = LST20_list(path = '/content/LST20_Corpus/eval/*')
val_data = get_data(val_df,
                    "POS",
                    BATCH_SIZE)
val_data.ADD_NER()

In [ ]:
test_df = LST20_list(path = '/content/LST20_Corpus/test/*')
test_data = get_data(test_df,
                    "POS",
                    BATCH_SIZE)
test_data.ADD_NER()

## Create Model

In [ ]:
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] 
unique_y = 16

In [ ]:
def LSTM_NER(vocab_len,emb_dim):
    WORD_inputs = layers.Input(shape=(BATCH_SIZE))
    NER_inputs = layers.Input(shape=(BATCH_SIZE))
    WORD_emb = layers.Embedding(trainable=False,input_dim = vocab_len, output_dim = emb_dim ,name="emb1")(WORD_inputs)
    NER_emb = layers.Embedding(trainable=True,input_dim = 40, output_dim = 768 ,name="emb2")(NER_inputs)
    chunk = tf.concat([WORD_emb,NER_emb],axis=2)
    blstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(chunk)
    blstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(unique_y, activation = 'softmax'))(x)

    act_model = Model([WORD_inputs,NER_inputs], x)
    act_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print(act_model.summary())
    return act_model

In [ ]:
model = LSTM_NER(vocab_len,emb_dim)

np_emb_weight = np_emb_weight.reshape(1,25005,768)
model.get_layer("emb1").set_weights(np_emb_weight) # use pretain weight

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 emb1 (Embedding)               (None, 128, 768)     19203840    ['input_7[0][0]']                
                                                                                                  
 emb2 (Embedding)               (None, 128, 768)     30720       ['input_8[0][0]']                
                                                                                            

In [ ]:
train_data.NER

array([[ 0.,  0.,  0., ...,  0.,  0.,  4.],
       [24.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  5., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
model.fit([train_data.x,train_data.NER],train_data.y, 
                   epochs=100,
                   #validation_data=([val_data.x,val_data.NER],val_data.y),
                   callbacks = [ES],
                   batch_size = 64)

Epoch 1/100
332/332 [==============================] - 21s 64ms/step - loss: 0.1051 - accuracy: 0.9638
Epoch 2/100
332/332 [==============================] - 21s 62ms/step - loss: 0.1008 - accuracy: 0.9649
Epoch 3/100
332/332 [==============================] - 21s 63ms/step - loss: 0.0980 - accuracy: 0.9656
Epoch 4/100
332/332 [==============================] - 21s 64ms/step - loss: 0.0949 - accuracy: 0.9664
Epoch 5/100
332/332 [==============================] - 21s 62ms/step - loss: 0.0923 - accuracy: 0.9672
Epoch 6/100
332/332 [==============================] - 21s 62ms/step - loss: 0.0898 - accuracy: 0.9679
Epoch 7/100
332/332 [==============================] - 21s 62ms/step - loss: 0.0880 - accuracy: 0.9682
Epoch 8/100
332/332 [==============================] - 21s 63ms/step - loss: 0.0857 - accuracy: 0.9689
Epoch 9/100
100/332 [========>.....................] - ETA: 14s - loss: 0.0826 - accuracy: 0.9698

KeyboardInterrupt: ignored

In [ ]:
model.evaluate([val_data.x,val_data.NER],val_data.y)

59/59 [==============================] - 5s 22ms/step - loss: 0.1029 - accuracy: 0.9637


[0.1029287651181221, 0.9636579751968384]

In [ ]:
pred = model.predict([test_data.x,test_data.NER])
pred = pred.argmax(axis=2)

In [ ]:
print(classification_report(pred.flatten(),test_data.y.flatten()))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     58310
         1.0       0.97      0.95      0.96     43385
         2.0       0.96      0.96      0.96     17631
         3.0       1.00      1.00      1.00     37977
         4.0       0.94      0.94      0.94     10946
         5.0       0.95      0.93      0.94      7643
         6.0       0.78      0.90      0.83      5820
         7.0       0.99      1.00      0.99      6904
         8.0       0.98      0.98      0.98      6260
         9.0       0.86      0.90      0.88      4201
        10.0       0.89      0.87      0.88      3834
        11.0       0.90      0.81      0.85      2390
        12.0       1.00      1.00      1.00      1687
        13.0       0.90      0.73      0.80       241
        14.0       0.00      0.00      0.00         0
        15.0       0.50      0.67      0.57         3

    accuracy                           0.96    207232
   macro avg       0.85   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# INPUT + NER

## data loader

In [ ]:
clean_data.shape

In [ ]:
clean_train,clean_val = train_test_split(clean_data,train_size=0.95,stratify=clean_data["POS"])

In [ ]:
clean_train.shape

In [ ]:
new_data = get_data(list(clean_train["text"]),list(clean_train["POS"]),BATCH_SIZE)

In [ ]:
clean_train["text"].shape

In [ ]:
new_val = get_data(list(clean_val["text"]),list(clean_val["POS"]),BATCH_SIZE)

In [ ]:
clean_train

## model

In [ ]:
def LSTM_new(vocab_len,emb_dim):
    inputs_WORD = layers.Input(shape=(BATCH_SIZE))
    inputs_NER = layers.Input(shape=(BATCH_SIZE))
    Emb_word = layers.Embedding(trainable=False,input_dim = vocab_len, output_dim = emb_dim ,name="emb")(inputs_WORD)
    Emb_NER = layers.Embedding(trainable=True,input_dim = vocab_len, output_dim = emb_dim ,name="emb")(inputs_NER)
    concat = tf.concat([Emb_word,Emb_NER],axis=2)
    blstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(concat)
    blstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(64, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(unique_y, activation = 'softmax'))(x)

    act_model = Model([inputs_WORD,inputs_NER], x)
    act_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print(act_model.summary())
    return act_model

In [ ]:
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] 
unique = train_data

In [ ]:
model_NER  = LSTM_new(vocab_len,emb_dim)

np_emb_weight = np_emb_weight.reshape(1,25005,768)
model_new.get_layer("emb").set_weights(np_emb_weight) # use pretain weight

In [ ]:
LSTM_NER = set_model()

# Bert

In [ ]:
from sklearn.metrics import classification_report

# After Model

## Dowload True Test

In [ ]:
!unzip /content/pos_test.txt.zip -d /content/submission/ 

In [ ]:
fp = open("/content/submission/pos_test.txt","r")

In [ ]:
word_list_submission = []
for line in fp.readlines() :
  if line != '\n' :
    word_list_submission.append(line[:-1])
  else :
    word_list_submission.append('_')

In [ ]:
word_list_submission = word_list_submission[:-1]
len(word_list_submission)

In [ ]:
last_pad = BATCH_SIZE - (len(word_list_submission)%BATCH_SIZE)

In [ ]:
word_list_submission += ['_']*last_pad

In [ ]:
word_list_submission_encoded = [endcode_word(r) for r in tqdm(word_list_submission)]

In [ ]:
np_word_list_submission_encoded_batched = np.array(word_list_submission_encoded).reshape(-1,BATCH_SIZE)

In [ ]:
np_word_list_submission_encoded_batched.shape

In [ ]:
word_list_submission

## Predict True test

In [ ]:
z = model.predict(np_word_list_submission_encoded_batched).argmax(axis=2)

In [ ]:
func2_ = np.vectorize(lambda x : decode_label(x))
z_decode = func2_(z)
z_decode = z_decode.flatten()
z_decode = z_decode.tolist()[:-last_pad]
print(len(z_decode))

## Submission

In [ ]:
!gdown --id 1mrbGe7J6nR1PZRGjCglZKugBghc_L-4_

In [ ]:
submission_df = pd.read_csv("/content/pos_sample_submission.csv")

In [ ]:
submission_df.Predicted = z_decode

In [ ]:
submission_df.to_csv("submission.csv",index=False)

In [ ]:
model.save('LSTM_emb_bert.h5')